# Experimenting rec approaches

In [1]:
import polars as pl
import duckdb

In [5]:
CUT_TIMESTAMP = '2018-01-01'

In [15]:
duckdb_conn = duckdb.connect('../data/steam.duckdb')

In [7]:
games_df =duckdb_conn.sql(f"SELECT * FROM game_rolling_features WHERE DATE(game_release_date) < '{CUT_TIMESTAMP}'").pl()
games_df.sort("game_index", descending=True)

game_index,game_id,game_name,game_developers,game_publishers,game_categories,game_genres,game_release_date,game_short_description,game_review_score,game_review_score_description,game_total_reviews,game_total_positive_reviews,game_total_negative_reviews,game_review_day,game_num_reviews,game_num_positive_reviews,game_num_negative_reviews,game_weighted_score
i64,i64,str,list[str],list[str],list[str],list[str],"datetime[μs, America/Lima]",str,i64,str,i64,i64,i64,date,i64,"decimal[38,0]","decimal[38,0]",f64
14244,595630,"""Neon8""","[""GameDevCoop""]","[""GameDevCoop""]","[""Single-player"", ""Full Controller Support"", … ""Family Sharing""]","[""Action""]",2017-12-31 00:00:00 -05,"""Neon8 is a fast-paced First Pe…",0,"""1 user reviews""",1,0,1,2018-01-15,1,0,1,0.0
14243,748650,"""Carnage in Space: Ignition""","[""MM//Productions""]","[""Throwback Entertainment""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",2017-12-31 00:00:00 -05,"""Use your fighting skill and su…",0,"""3 user reviews""",3,2,1,2020-01-24,3,2,1,0.754363
14243,748650,"""Carnage in Space: Ignition""","[""MM//Productions""]","[""Throwback Entertainment""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",2017-12-31 00:00:00 -05,"""Use your fighting skill and su…",0,"""3 user reviews""",3,2,1,2018-03-17,2,2,0,1.0
14243,748650,"""Carnage in Space: Ignition""","[""MM//Productions""]","[""Throwback Entertainment""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",2017-12-31 00:00:00 -05,"""Use your fighting skill and su…",0,"""3 user reviews""",3,2,1,2018-02-28,1,1,0,1.0
14242,764180,"""Cross of Auria""","[""Hardytier""]","[""Hardytier""]","[""Single-player"", ""Steam Achievements"", … ""Steam Cloud""]","[""Indie"", ""RPG""]",2017-12-31 00:00:00 -05,"""Cross of Auria is primarily a …",0,"""7 user reviews""",7,6,1,2020-01-19,5,4,1,0.827533
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,232770,"""POSTAL""","[""Running With Scissors""]","[""Running With Scissors""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",1997-11-14 00:00:00 -05,"""Take control of The POSTAL Dud…",8,"""Very Positive""",1323,1144,179,2018-02-08,769,653,116,0.86319
0,232770,"""POSTAL""","[""Running With Scissors""]","[""Running With Scissors""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",1997-11-14 00:00:00 -05,"""Take control of The POSTAL Dud…",8,"""Very Positive""",1323,1144,179,2018-03-18,784,667,117,0.864824
0,232770,"""POSTAL""","[""Running With Scissors""]","[""Running With Scissors""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",1997-11-14 00:00:00 -05,"""Take control of The POSTAL Dud…",8,"""Very Positive""",1323,1144,179,2020-05-17,1003,861,142,0.872272


In [8]:
dim_games_df =duckdb_conn.sql(f"SELECT * FROM dim_games_imputed WHERE DATE(game_release_date) < '{CUT_TIMESTAMP}'").pl()
dim_games_df.sort("game_index", descending=True)

game_index,game_id,game_name,game_developers,game_publishers,game_categories,game_genres,game_release_date,game_short_description,game_review_score,game_review_score_description,game_total_reviews,game_total_positive_reviews,game_total_negative_reviews
i64,i64,str,list[str],list[str],list[str],list[str],"datetime[μs, America/Lima]",str,i64,str,i64,i64,i64
14247,667590,"""Fjord battle racing""","[""Gaute Heggen""]","[""Cannon head interactive""]","[""Multi-player"", ""PvP"", … ""Family Sharing""]","[""Racing"", ""Early Access""]",2017-12-31 00:00:00 -05,"""Local multiplayer battle racin…",0,"""No reviews""",0,0,0
14246,717990,"""Bloodworks""","[""Engin Mercan""]","[""Engin Mercan""]","[""Single-player"", ""Steam Achievements"", … ""Controller enabled""]","[""Action"", ""Casual"", … ""Indie""]",2017-12-31 00:00:00 -05,"""Bloodworks is a free top-down …",0,"""No reviews""",0,0,0
14245,735250,"""BLACKSTONE""","[""Fire Engrave""]","[""Fire Engrave""]","[""Single-player"", ""Controller enabled"", ""Family Sharing""]","[""RPG""]",2017-12-31 00:00:00 -05,"""BLACKSTONE is a role-playing g…",0,"""No reviews""",0,0,0
14244,595630,"""Neon8""","[""GameDevCoop""]","[""GameDevCoop""]","[""Single-player"", ""Full Controller Support"", … ""Family Sharing""]","[""Action""]",2017-12-31 00:00:00 -05,"""Neon8 is a fast-paced First Pe…",0,"""1 user reviews""",1,0,1
14243,748650,"""Carnage in Space: Ignition""","[""MM//Productions""]","[""Throwback Entertainment""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Indie""]",2017-12-31 00:00:00 -05,"""Use your fighting skill and su…",0,"""3 user reviews""",3,2,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
4,10,"""Counter-Strike""","[""Valve""]","[""Valve""]","[""Multi-player"", ""PvP"", … ""Family Sharing""]","[""Action""]",2000-11-01 00:00:00 -05,"""Play the world's number 1 onli…",9,"""Overwhelmingly Positive""",32951,31837,1114
3,50,"""Half-Life: Opposing Force""","[""Gearbox Software""]","[""Valve""]","[""Single-player"", ""Multi-player"", … ""Family Sharing""]","[""Action""]",1999-11-01 00:00:00 -05,"""Return to the Black Mesa Resea…",9,"""Overwhelmingly Positive""",11992,11438,554
2,20,"""Team Fortress Classic""","[""Valve""]","[""Valve""]","[""Multi-player"", ""PvP"", … ""Family Sharing""]","[""Action""]",1999-04-01 00:00:00 -05,"""One of the most popular online…",8,"""Very Positive""",4018,3469,549


In [12]:
duckdb_conn.sql(f"SELECT * FROM dim_games_imputed WHERE game_index = 73131").pl()

game_index,game_id,game_name,game_developers,game_publishers,game_categories,game_genres,game_release_date,game_short_description,game_review_score,game_review_score_description,game_total_reviews,game_total_positive_reviews,game_total_negative_reviews
i64,i64,str,list[str],list[str],list[str],list[str],"datetime[μs, America/Lima]",str,i64,str,i64,i64,i64
73131,394690,"""Tower Unite""","[""PixelTail Games""]","[""PixelTail Games""]","[""Single-player"", ""Multi-player"", … ""Family Sharing""]","[""Action"", ""Adventure"", … ""Racing""]",2025-04-19 00:00:00 -05,"""PLAY, CREATE, PARTY! Tower Uni…",8,"""Very Positive""",9985,9014,971


In [9]:
reviews_df = duckdb_conn.sql(f"SELECT * FROM fact_reviews WHERE DATE(timestamp_created) < '{CUT_TIMESTAMP}'").pl()
reviews_df.sort("game_index", descending=True)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

user_index,game_index,review,voted_up,weighted_vote_score,votes_up,comment_count,timestamp_created
i64,i64,str,bool,f64,i64,i64,"datetime[μs, America/Lima]"
925490,74914,"""Spectacular graphics but like …",true,0.52381,7,1,2017-11-02 02:41:13 -05
1488980,74914,"""This table is fantastic! Good…",true,0.703571,22,0,2017-10-14 12:57:21 -05
2313147,74914,"""This game is pretty fun. It ha…",true,0.47619,4,1,2017-12-18 20:54:03 -05
1936139,74914,"""This game in it's current stat…",false,0.363442,13,10,2017-10-14 01:14:00 -05
1954681,74914,"""Decided to give it a try Nov 2…",true,0.540918,6,2,2017-11-23 06:22:56 -05
…,…,…,…,…,…,…,…
2308931,0,"""2D 3D who care's postal game w…",true,0.579802,9,0,2017-11-24 14:00:13 -05
2301001,0,"""The developers are still activ…",true,0.625752,8,0,2017-12-09 18:38:11 -05
931300,0,"""One of the most disturbing and…",true,0.573754,9,1,2017-11-13 22:16:35 -05


In [ ]:
inconsistent_release_dates = duckdb_conn.sql(
    f"SELECT g.game_id, g.game_index, g.game_name, g.game_release_date, f.timestamp_created FROM dim_games_imputed g JOIN fact_reviews f ON f.game_index = f.game_index WHERE DATE(f.timestamp_created) < DATE(g.game_release_date)"
).pl()
inconsistent_release_dates

In [13]:
duckdb_conn.close()